# Spark SQL

This notebook introduces how to perform basic data transformation and exploration using Spark SQL on [Avocado Prices dataset](https://www.kaggle.com/datasets/neuromusic/avocado-prices) in Kaggle.

## Spark Cluster Preparation

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [ ]:
import findspark
findspark.init()

In [ ]:
spark_url = 'local'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

## Spark SQL Data Preparation

First, we read a csv file.  We can provide option such as delimiter and header.  We then rename the colume names to remove dot ('.') in the names.

In [ ]:
path = 'avocado.csv'

In [ ]:
df = spark.read.csv(path, header=True, inferSchema=True)

Check out data and schema

In [ ]:
df.printSchema()

Those columns with spaces or dots in the column names will be a bit difficult for Spark SQL, esepcially when we want to filter data.  We must first remove spaces/dots in the column names.  This is optional, though.

In [ ]:
cols = [c.replace(' ', '_') for c in df.columns]
df = df.toDF(*cols)

In [ ]:
df.columns

In [ ]:
df.show(5)

Using *'inferSchema=True'*, Spark SQL seems to guess datatype correctly.  However, if it is incorrect, we can cast each column to proper type using **'cast'** and replace back to the same column using **'withColumn'**.

## Basic Spark SQL Commands

We can select some columns using **'select'** and select some rows using **'filter'**.  Note that we can perform basic math to columns.

In [ ]:
df.select(df['Date'], df['AveragePrice'], df['Total_Bags'], df['year'], df['region']).show(5)

In [ ]:
df.select(df['Date'], df['Small_Bags']+df['Large_Bags']+df['XLarge_Bags'], 
          df['Total_Bags']).show(5)

In [ ]:
df.select(df['Date'], df['Total_Bags'], df['Total_Volume'], 
          df['Total_Volume']/df['Total_Bags']).show(5)

In [ ]:
df.filter(df['Total_Bags'] < 8000).show(3)

In [ ]:
df.filter((df['Total_Bags'] < 8000) & (df.year > 2015)).select('Date', 'Total_Bags').show(3)

In [ ]:
df.filter('Total_Bags > 8000 and year > 2015').select('Date', 'Total_Bags').show(3)

In [ ]:
df.filter('region == "SanDiego"').show(3)

## Aggregate and Groupby Functions
We can use several built-in aggegrate functions.  We can also use groupby for group operations

### Explore Distinct Values

Let's explore those categorical data including region and type using select-distinct.  Note that it can be applied to those numerical data too

In [ ]:
df.select('type').distinct().show()

In [ ]:
df.select('region').distinct().show()

In [ ]:
df.select('year').distinct().show()

### Using group and groupby functions

In [ ]:
from pyspark.sql.functions import avg, min, max

In [ ]:
df.select(min('AveragePrice'), avg('AveragePrice'), max('AveragePrice')).show()

In [ ]:
df.filter('region == "SanDiego"').select(avg('AveragePrice')).show()

Groupby function allows us to work data in groups.

In [ ]:
df.groupby('type').count().show()

In [ ]:
df.groupby('year', 'type').agg({'AveragePrice': 'avg'}).orderBy('year', 'type').show()

## User-Defined Function
We can create user-defined function using udf.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [ ]:
def pricegroup_mapping(price): 
    if price < 1:
        return 'cheap'
    if price < 2:
        return 'moderate'
    return 'expensive'

to_pricegroup = udf(pricegroup_mapping, StringType())

In [ ]:
df.select('Date', 'AveragePrice', to_pricegroup('AveragePrice')).show(5)

Using *'withColumn'*, we can compute values and store them to the new column

In [ ]:
new_df = df.withColumn('pricegroup', to_pricegroup(df.AveragePrice))
new_df.select('AveragePrice', 'pricegroup').show(5)

## Pyspark Pandas API
Since release 3.2, Spark provides a Pandas API that enables users to use Pandas-compatible command with just one import (pyspark.pandas).  This lowers learning-curve and enables other features e.g. plotting graph.

In [ ]:
import pyspark.pandas as ps

In [ ]:
pdf = ps.DataFrame(df)

In [ ]:
pdf.head()

In [ ]:
pdf.AveragePrice.plot.box()

In [ ]:
counts = pdf.groupby('year').Date.count()

In [ ]:
counts

In [ ]:
counts.plot.pie()

In [ ]:
spark.stop()